In [59]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json
import numpy as np
from itertools import chain
from yarl import URL

**TODO**:

- `ruff` $\Rightarrow$ linter, à utiliser plus dans le cadre du dvpt
- `pytest` avec subprocess.run pour tester le code
- `pytest-coverage`
- `poetry` avec gestion des dépendances
- `black` pour automatiquement indenter, etc.
- `doctest` pour tester les exemples des docstrings
- `bandit` pour gérer les vulnérabilités

***

- `yarl` pour travailler avec des URL

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [60]:
URL()

URL('')

In [61]:
session = create_session()
url_list = create_url_list(URL, 1)
pages = read_pages(url_list=url_list, session=session)

In [ ]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

Elapsed time for extract_all_urls function: 0.488 seconds.

--------------------
Product 1 -> Successfull Extraction.
Product 2 -> Successfull Extraction.
Product 3 -> Successfull Extraction.
Product 4 -> Successfull Extraction.
Product 5 -> Successfull Extraction.
Product 6 -> Successfull Extraction.
Product 7 -> Successfull Extraction.
Product 8 -> Successfull Extraction.
Product 9 -> Successfull Extraction.
Product 10 -> Successfull Extraction.
Product 11 -> Successfull Extraction.
Product 12 -> Successfull Extraction.
Product 13 -> Successfull Extraction.
Product 14 -> Successfull Extraction.
Product 15 -> Successfull Extraction.
Product 16 -> Successfull Extraction.
Product 17 -> Successfull Extraction.
Product 18 -> Successfull Extraction.
Product 19 -> Successfull Extraction.
Product 20 -> Successfull Extraction.
Product 21 -> Successfull Extraction.
Product 22 -> Successfull Extraction.
Product 23 -> Successfull Extraction.
Product 24 -> Successfull Extraction.
Product 25 -> Successfull Extraction.
Product 26 -> Successfull Extraction.


Elapsed time for extract_all_pages function: 59.688 seconds.

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [ ]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [ ]:
smartphones_json = to_json(smartphone_list)

In [ ]:
file_path = "data/smartphones.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [1]:
import polars as pl
import polars.selectors as cs
from script.polars_gazprom import *

In [2]:
df_brut = pl.read_json("data/smartphones.json")

In [ ]:
df_brut.select(pl.all().is_null().sum())

### Pipeline

Sur `535` extractions, il y a `436` téléphones valides avec les caractéristiques. Pas mal ?

Pour obtenir la *PPI* (Pixels per Inch) on fait : 

$$PPI = \dfrac{\text{diagonal\_pixels}}{\text{screen\_size} (inch)}$$

In [3]:
df = NordStream(df_brut)

In [50]:
#df.select(cs.by_dtype(pl.NUMERIC_DTYPES)).corr()

# Matrice de corrélation

In [6]:
import statsmodels.formula.api as smf
import statsmodels.formula as sm
from stargazer.stargazer import Stargazer
import numpy as np
from dataclasses import dataclass
from script.hedonic_regression import *

In [8]:
df

model,net_weight,price,upgrade_storage,made_in,color,sensor,width,brand,usb_type_c,repairability_index,battery,cpu,thickness,screen_size,das_head,das_chest,das_limbs,height,stars,screen_tech,reviews,network,storage,fast_charging,screen_type,ram,cam_1,cam_2,cam_3,diagonal_pixels,logprice,induction,mpx_backward_cam,random_col
str,f64,f64,bool,str,str,str,f64,str,bool,f64,i64,str,f64,f64,f64,f64,f64,f64,f64,str,i64,str,i64,bool,str,i64,f64,f64,f64,f64,f64,bool,f64,i64
"""SONY Xperia 5 …",0.0,999.0,true,"""Thaïlande""","""Argent""","""3""",0.7,"""SONY""",true,8.5,5000,"""Qualcomm Snapd…",0.9,6.1,0.95,1.05,2.11,15.4,0.0,"""OLED FHD+""",0,"""5G""",128,true,"""Plat""",8,12.0,12.0,12.0,2741.678318,6.906755,true,36.0,413
"""Samsung Galaxy…",167.0,857.59,false,"""Viêt Nam""","""Noir""","""3""",87.0,"""SAMSUNG""",true,8.2,3700,"""Exynos 2200""",0.8,6.1,1.21,1.59,2.71,146.0,4.0,"""Dynamic AMOLED…",1,"""5G""",256,true,"""Plat""",8,50.0,12.0,10.0,2577.207791,6.754126,true,72.0,-582
"""Motorola G53""",183.0,199.0,true,"""Chine""","""Bleu""","""2""",74.7,"""MOTOROLA""",true,7.1,5000,"""8 coeurs jusqu…",8.2,6.5,0.98,1.32,2.98,162.7,4.3333,"""LCD""",3,"""5G""",128,false,"""Plat""",4,50.0,2.0,0.0,1754.536976,5.293305,false,52.0,-511
"""Google Pixel 8…",null,799.0,false,"""Viêt Nam""","""Noir""","""3""",7.1,"""GOOGLE""",true,8.2,4500,"""Google Tensor …",0.9,6.2,0.98,1.2,2.49,15.1,0.0,"""Actua ou OLED …",0,"""5G""",128,true,"""Plat""",8,50.0,12.0,10.0,2631.805464,6.683361,true,72.0,185
"""Xiaomi Redmi 9…",194.0,95.9,true,"""Chine""","""Gris""","""1""",77.1,"""XIAOMI""",true,7.0,5000,"""8 coeurs jusqu…",9.0,6.5,0.398,1.186,2.526,164.9,4.0,"""HD+""",2,"""4G""",32,true,"""Plat""",2,13.0,0.0,0.0,1754.536976,4.563306,false,13.0,613
"""HONOR Magic5 L…",175.0,299.0,false,"""Chine""","""Noir""","""3""",73.9,"""HONOR""",true,8.0,5100,"""Qualcomm Snapd…",7.9,6.7,0.82,1.27,2.81,161.6,4.1333,"""AMOLED (rigide…",15,"""5G""",256,true,"""Bords incurvés…",8,64.0,5.0,2.0,2631.805464,5.700444,false,71.0,600
"""Google Pixel 6…",210.0,633.67,false,"""Chine""","""Noir""","""3""",75.9,"""GOOGLE""",true,6.4,5000,"""Google Tensor""",8.9,6.7,0.99,1.4,2.96,163.9,4.087,"""Non communiqué…",23,"""5G""",128,true,"""Plat""",12,50.0,48.0,12.0,3436.277055,6.451528,true,110.0,989
"""Realme GT Mast…",174.0,304.25,true,"""Chine""","""Noir""",null,73.5,"""REALME""",true,6.2,4300,"""Non communiqué…",8.0,6.4,1.139,1.18,1.88,159.1,0.0,"""Non communiqué…",0,"""5G""",128,true,"""Borderless""",6,0.0,0.0,0.0,2631.805464,5.71785,false,0.0,-192
"""Google Pixel 8…",null,859.0,false,"""Viêt Nam""","""Vert""","""3""",7.1,"""GOOGLE""",true,8.2,4500,"""Google Tensor …",0.9,6.2,0.98,1.2,2.49,15.1,0.0,"""Actua ou OLED …",0,"""5G""",256,true,"""Plat""",8,50.0,12.0,10.0,2631.805464,6.755769,true,72.0,-801


In [20]:
y_1 = "price"
y_2 = "logprice"
X = [
    "storage",
    "induction",
    "upgrade_storage",
    "ram",
    "screen_type",
    "network",
    "total_pixels",
    "screen_size",
    "brand",
    "made_in",
    "das_limbs",
    "battery"
]

In [22]:
f_1 = formula(y_1, X)
f_2 = formula(y_2, X)
res_1 = execute_ols(df, f_1)
res_2 = execute_ols(df, f_2)

In [24]:
Stargazer([res_1, res_2])

Variables qui n'apportent rien dans la loglin-reg : 

- color
- net_weight
- model
- das_chest
- das_head

In [223]:
def feature_importance(df: pl.DataFrame, y:str):
    column_name = list()
    r2_list = list()
    for column in df.columns:
        ols_model = smf.ols(
        formula=f"{y}~{column}",
        data=df)
        res = ols_model.fit()
        column_name.append(column)
        r2_list.append(res.rsquared)

    feature_imp_df = pl.DataFrame({'Column': column_name, 'R-squared': r2_list}).sort("R-squared", descending=True)
    return feature_imp_df

In [224]:
df_features = feature_importance(df, "price")

In [199]:
import plotly.express as px

In [225]:
fig = px.bar(df_features, x='Column', y='R-squared', title='Feature Importance (R-squared)')
fig.update_xaxes(title_text='Feature')
fig.update_yaxes(title_text='R-squared')
fig.show()

In [ ]:
def predict_phone_price(df: pl.DataFrame, f: Formula, storage: float, ram: float):
    res = execute_ols(df, f)
    params = dict(res.params)
    prediction = params["Intercept"] + params["storage"]*storage + params["ram"]*ram
    return prediction

In [ ]:
predict_phone_price(df, f, 32, 1)

143.2669566783436

In [ ]:
ols_model = smf.ols(
    formula=f"{y}~{X2}",
    data=df,
)

In [ ]:
res = ols_model.fit()

In [ ]:
dict(res.params)

{'Intercept': 1071.5195276459249,
 'storage': 1.1958931708843838,
 'ram': 53.85417103504183,
 'battery': -0.23112324797420558}

In [ ]:
real_price = df.select("price").to_numpy()

In [ ]:
price_predict = res.predict()

In [ ]:
# res2 = res.get_robustcov_results(cov_type='HC1')
# robuste std errors

In [66]:
Stargazer([res])

- fuzzy match à mettre en place ici (pour la technologie de l'écran) ?
- ajouter une feature complètement random (pour la partie Machine Learning) puis regarder l'importance des variables : si les variables sont moins importantes que la random, alors on les dégage

In [ ]:
df.group_by("brand").count().sort("count")

brand,count
str,u32
"""FAIRPHONE""",3
"""ONEPLUS""",7
"""NOTHING""",7
"""VIVO""",9
"""ASUS""",12
"""REALME""",14
"""HONOR""",14
"""SONY""",15
"""OPPO""",18


**TODO** `cleaning` :

- [x] **network**
- [x] **color**
- [x] **price**
- [x] **screen_type**
- [ ] *cpu*

In [ ]:
# df.select("cpu").unique().to_numpy()
# pour le cpu on va préférer retravailler pour avoir la fréquence en GHz plutot

Vérification des valeurs nulles sur les différentes colonnes.

In [ ]:
df.select(pl.all().is_null().sum())

screen_tech,upgrade_storage,height,cpu,reviews,storage,color,fast_charging,sensor,model,sensor_resolution,ram,charging_type,repairability_index,das_chest,screen_size,network,net_weight,stars,das_limbs,battery,made_in,price,width,sim_type,thickness,das_head,screen_type,usb_type_c,brand,diagonal_pixels
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,1,1,0,0,1,0,75,0,0,0,3,0,0,0,0,14,0,0,0,7,0,1,0,1,0,0,0,0,0
